Aşağıda 3 farklı kullanıcının sepet bilgileri verilmiştir. Bu sepet bilgilerine en uygun ürün önerisini birliktelik kuralı kullanarak yapınız. Karar kurallarını 2010-2011 Germany müşterileri üzerinden türetiniz.

Kullanıcı 1’in sepetinde bulunan ürünün id'si: 21987

Kullanıcı 2’in sepetinde bulunan ürünün id'si : 23235

Kullanıcı 3’in sepetinde bulunan ürünün id'si : 22747

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df = pd.read_excel("C:/Users/htcso/OneDrive/Masaüstü/pythonProject/datasets/online_retail_II.xlsx",
                sheet_name = "Year 2010-2011")

In [3]:
retail = df.copy()

In [4]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
retail.shape

(541910, 8)

In [6]:
german_retail = retail[retail["Country"] == "Germany"]

In [7]:
german_retail.shape

(9495, 8)

In [8]:
german_retail = german_retail[german_retail["StockCode"] != "POST"]

In [9]:
german_retail.shape

(9112, 8)

In [10]:
german_retail.dropna(inplace = True)

In [11]:
german_retail.shape

(9112, 8)

In [12]:
german_retail = german_retail[~german_retail["Invoice"].str.contains("C", na=False)]

In [13]:
german_retail.shape

(8668, 8)

In [14]:
german_retail = german_retail[german_retail["Price"] > 0 ]
german_retail = german_retail[german_retail["Quantity"] > 0 ]

In [15]:
german_retail.shape

(8666, 8)

In [16]:
def outlier_threshold(dataframe, variable):
    q1 = dataframe[variable].quantile(0.01)
    q3 = dataframe[variable].quantile(0.99)
    iqr = q3 - q1
    up_limit = q3 + 1.5 * iqr
    low_limit = q1 - 1.5 * iqr
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_threshold(dataframe, variable)
    dataframe.loc[dataframe[variable] > up_limit, variable] = up_limit
    dataframe.loc[dataframe[variable] < low_limit, variable] = low_limit

In [17]:
replace_with_thresholds(german_retail, "Price")
replace_with_thresholds(german_retail, "Quantity")

In [18]:
invoice_product_df = german_retail.groupby(["Invoice", "StockCode"])["Quantity"].sum().\
    unstack().\
    fillna(0).\
    applymap(lambda x: 1 if x > 0 else 0)

invoice_product_df.head()

StockCode,10002,10125,10135,11001,15034,15036,15039,16008,16011,16014,...,90160D,90161B,90161C,90161D,90201A,90201B,90201C,90201D,90202D,M
Invoice,,,,,,,,,,,,,,,,,,,,,
536527,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536840,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536861,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536983,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
def create_rules(df, metric="support", min_th=0.01):
    frequent_itemsets = apriori(df, min_support=min_th, use_colnames=True)
    rules = association_rules(frequent_itemsets, min_threshold=min_th, metric=metric)
    return rules

In [20]:
rules = create_rules(invoice_product_df)
rules.head()

C:\Users\htcso\anaconda3\lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(16237),(22326),0.011136,0.249443,0.011136,1.000000,4.008929,0.008358,inf
1,(22326),(16237),0.249443,0.011136,0.011136,0.044643,4.008929,0.008358,1.035073
2,(20674),(20675),0.022272,0.033408,0.013363,0.600000,17.960000,0.012619,2.416481
3,(20675),(20674),0.033408,0.022272,0.013363,0.400000,17.960000,0.012619,1.629547
4,(20674),(20676),0.022272,0.037862,0.011136,0.500000,13.205882,0.010293,1.924276


In [21]:
def check_stock_code(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code]["Description"].values[0]
    print(product_name)

In [22]:
user1_product_id = 21987
user2_product_id = 23235
user3_product_id = 22747

check_stock_code(retail, user1_product_id)
check_stock_code(retail, user2_product_id)
check_stock_code(retail, user3_product_id)

PACK OF 6 SKULL PAPER CUPS
STORAGE TIN VINTAGE LEAF
POPPY'S PLAYHOUSE BATHROOM


In [23]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])
    return recommendation_list[:rec_count]

In [24]:
print(arl_recommender(rules, user1_product_id))
print(arl_recommender(rules, user2_product_id))
print(arl_recommender(rules, user3_product_id))

[21989]
[23244]
[22746]


In [25]:
check_stock_code(retail, 21989)
check_stock_code(retail, 23244)
check_stock_code(retail, 22746)

PACK OF 20 SKULL PAPER NAPKINS
ROUND STORAGE TIN VINTAGE LEAF
POPPY'S PLAYHOUSE LIVINGROOM 
